In [143]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [144]:
df = pd.read_csv('AirbnbMadrid_Listings.csv', encoding='latin-1')
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20751 entries, 0 to 20750
Data columns (total 58 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0.1                    20751 non-null  int64  
 1   Unnamed: 0                      20751 non-null  int64  
 2   id                              20751 non-null  float64
 3   nombre                          20751 non-null  object 
 4   descripcion                     20751 non-null  object 
 5   id_anfitrion                    20751 non-null  int64  
 6   nombre_anfitrion                20751 non-null  object 
 7   anfitrion_desde                 20751 non-null  object 
 8   ubicacion_anfitrion             20751 non-null  object 
 9   tiempo_respuesta_anfitrion      20751 non-null  object 
 10  tasa_respuesta_anfitrion        20751 non-null  float64
 11  tasa_aceptacion_anfitrion       20751 non-null  float64
 12  es_superanfitrion               

In [145]:
# Copia de seguridad del DataFrame original
df_log = df.copy()

#  tiempo_respuesta_anfitrion
df_log['tiempo_respuesta_bin'] = np.where(
    df_log['tiempo_respuesta_anfitrion'].isin(['within an hour', 'within a few hours']),
    1,  # Rápida
    0   # Lenta
)

#  tasa_respuesta_anfitrion
df_log['tasa_respuesta_bin'] = np.where(
    df_log['tasa_respuesta_anfitrion'] >= 70,
    1,  # Alta
    0   # Media - Baja
)

#  tasa_aceptacion_anfitrion
df_log['tasa_aceptacion_bin'] = np.where(
    df_log['tasa_aceptacion_anfitrion'] >= 70,
    1,  # Alta
    0   # Media - Baja
)

#  es_superanfitrion
df_log['superanfitrion_bin'] = np.where(
    df_log['es_superanfitrion'].str.strip().str.lower() == 'si',
    1,  # Sí es superanfitrión
    0   # No
)

#  distrito
df_log['distrito_bin'] = np.where(
    df_log['distrito'].str.strip().str.lower() == 'centro',
    1,  # Centro
    0   # Limítrofe
)

#  capacidad
df_log['capacidad_bin'] = np.where(
    df_log['capacidad'] >= 4,
    1,  # 4 o más
    0   # 3 o menos
)

#  review_scores_rating
df_log['rating_bin'] = np.where(
    df_log['review_scores_rating'] >= 4,
    1,  # Buena
    0   # Regular - Mala
)

#  review_scores_accuracy
df_log['accuracy_bin'] = np.where(
    df_log['review_scores_accuracy'] >= 4,
    1,  # Buena
    0   # Regular - Mala
)

#  review_scores_cleanliness
df_log['cleanliness_bin'] = np.where(
    df_log['review_scores_cleanliness'] >= 4,
    1,  # Buena
    0   # Regular - Mala
)

#  review_scores_location
df_log['location_bin'] = np.where(
    df_log['review_scores_location'] >= 4,
    1,  # Buena
    0   # Regular - Mala
)

#  Verificación rápida
cols_bin = [
    'tiempo_respuesta_bin', 'tasa_respuesta_bin', 'tasa_aceptacion_bin',
    'superanfitrion_bin', 'distrito_bin', 'capacidad_bin',
    'rating_bin', 'accuracy_bin', 'cleanliness_bin', 'location_bin'
]

print("Distribución de cada variable binaria:")
for col in cols_bin:
    print(f"{col} -> {df_log[col].value_counts(normalize=True).round(2)}")

Distribución de cada variable binaria:
tiempo_respuesta_bin -> tiempo_respuesta_bin
1    0.78
0    0.22
Name: proportion, dtype: float64
tasa_respuesta_bin -> tasa_respuesta_bin
1    0.95
0    0.05
Name: proportion, dtype: float64
tasa_aceptacion_bin -> tasa_aceptacion_bin
1    0.87
0    0.13
Name: proportion, dtype: float64
superanfitrion_bin -> superanfitrion_bin
0    0.72
1    0.28
Name: proportion, dtype: float64
distrito_bin -> distrito_bin
0    0.57
1    0.43
Name: proportion, dtype: float64
capacidad_bin -> capacidad_bin
0    0.6
1    0.4
Name: proportion, dtype: float64
rating_bin -> rating_bin
1    0.97
0    0.03
Name: proportion, dtype: float64
accuracy_bin -> accuracy_bin
1    0.97
0    0.03
Name: proportion, dtype: float64
cleanliness_bin -> cleanliness_bin
1    0.97
0    0.03
Name: proportion, dtype: float64
location_bin -> location_bin
1    0.98
0    0.02
Name: proportion, dtype: float64


tiempo_respuesta

In [146]:
# VarDep: tiempo_respuesta_bin
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['tasa_respuesta_anfitrion']]
Var_Dep= df_log['tiempo_respuesta_bin']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [147]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array([1, 1, 1, ..., 1, 1, 1], shape=(6226,))

In [148]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 351  990]
 [ 210 4675]]


In [149]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision1 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision1)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud1 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud1)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad1 = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad1)

Precisión del modelo:
0.8252427184466019
Exactitud del modelo:
0.807259877931256
Sensibilidad del modelo:
0.26174496644295303


tasa_respuesta

In [150]:
# VarDep:'tasa_respuesta_bin
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['tasa_aceptacion_anfitrion']]
Var_Dep= df_log['tasa_respuesta_bin']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [151]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 139  186]
 [  74 5827]]


In [152]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision2 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision2)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud2 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud2)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad2 = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad2)

Precisión del modelo:
0.9690670214535174
Exactitud del modelo:
0.9582396402184388
Sensibilidad del modelo:
0.4276923076923077


tasa_aceptacion

In [153]:
# VarDep:'tasa_aceptacion_bin
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['tasa_respuesta_anfitrion']]
Var_Dep= df_log['tasa_aceptacion_bin']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [154]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 214  592]
 [  45 5375]]


In [155]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision3 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision3)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud3 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud3)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad3 = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad3)

Precisión del modelo:
0.9007876654935478
Exactitud del modelo:
0.897687118535175
Sensibilidad del modelo:
0.2655086848635236


es_superanfitrion

In [156]:
# VarDep:'superanfitrion_bin
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['tasa_respuesta_anfitrion', 'tasa_aceptacion_anfitrion', 'review_scores_rating', 'ingresos_estimados_365d']]
Var_Dep= df_log['superanfitrion_bin']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [157]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[4361  177]
 [1533  155]]


In [158]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision4 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision4)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud4 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud4)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad4 = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad4)

Precisión del modelo:
0.46686746987951805
Exactitud del modelo:
0.7253453260520398
Sensibilidad del modelo:
0.9609960334949317


Distrito

In [159]:
# VarDep:'distrito_bin
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['review_scores_location', 'ingresos_estimados_365d']]
Var_Dep= df_log['distrito_bin']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [160]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[3054  549]
 [1845  778]]


In [161]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision5 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision5)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud5 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud5)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad5 = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad5)

Precisión del modelo:
0.586284853051997
Exactitud del modelo:
0.6154834564728557
Sensibilidad del modelo:
0.8476269775187344


Capacidad

In [162]:
# VarDep:'capacidad_bin
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['camas', 'habitaciones']]
Var_Dep= df_log['capacidad_bin']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [163]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[3423  326]
 [ 928 1549]]


In [164]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision6 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision6)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud6 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud6)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad6 = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad6)

Precisión del modelo:
0.8261333333333334
Exactitud del modelo:
0.7985865724381626
Sensibilidad del modelo:
0.9130434782608695


review_scores_rating

In [165]:
# VarDep:'rating_bin
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['review_scores_value']]
Var_Dep= df_log['rating_bin']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [166]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 109   70]
 [  20 6027]]


In [167]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision7 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision7)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud7 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud7)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad7 = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad7)

Precisión del modelo:
0.9885189437428243
Exactitud del modelo:
0.9855444908448442
Sensibilidad del modelo:
0.6089385474860335


Reponderación de clases 

In [168]:
#Reponderación de las clases
algoritmo_Pond = LogisticRegression(class_weight='balanced')
algoritmo_Pond.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [169]:
#Realizamos una predicción
y_pred_pond = algoritmo_Pond.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred_pond)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 174    5]
 [ 343 5704]]


In [170]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision7b = precision_score(y_test, y_pred_pond, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision7b)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud7b = accuracy_score(y_test, y_pred_pond)
print('Exactitud del modelo:')
print(exactitud7b)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad7b = recall_score(y_test, y_pred_pond, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad7b)

Precisión del modelo:
0.3365570599613153
Exactitud del modelo:
0.9441053646000642
Sensibilidad del modelo:
0.9432776583429799


review_scores_accuracy

In [171]:
# VarDep:'accuracy_bin
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['review_scores_rating']]
Var_Dep= df_log['accuracy_bin']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [172]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  81   71]
 [  22 6052]]


In [173]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision8 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision8)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud8 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud8)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad8 = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad8)

Precisión del modelo:
0.9884043769394087
Exactitud del modelo:
0.9850626405396723
Sensibilidad del modelo:
0.5328947368421053


Reponderación de clases 

In [174]:
#Reponderación de las clases
algoritmo_Pond = LogisticRegression(class_weight='balanced')
algoritmo_Pond.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [175]:
#Realizamos una predicción
y_pred_pond = algoritmo_Pond.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred_pond)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 144    8]
 [ 313 5761]]


In [176]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision8b = precision_score(y_test, y_pred_pond, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision8b)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud8b = accuracy_score(y_test, y_pred_pond)
print('Exactitud del modelo:')
print(exactitud8b)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad8b = recall_score(y_test, y_pred_pond, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad8b)

Precisión del modelo:
0.3150984682713348
Exactitud del modelo:
0.9484420173466109
Sensibilidad del modelo:
0.9484688837668752


review_scores_cleanliness

In [177]:
# VarDep:'cleanliness_bin
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['review_scores_rating']]
Var_Dep= df_log['cleanliness_bin']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [178]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  60  110]
 [  34 6022]]


In [179]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision9 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision9)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud9 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud9)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad9 = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad9)

Precisión del modelo:
0.9820613176777561
Exactitud del modelo:
0.9768711853517508
Sensibilidad del modelo:
0.35294117647058826


Reponderación de clases 

In [180]:
#Reponderación de las clases
algoritmo_Pond = LogisticRegression(class_weight='balanced')
algoritmo_Pond.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [181]:
#Realizamos una predicción
y_pred_pond = algoritmo_Pond.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred_pond)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 153   17]
 [ 479 5577]]


In [182]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision9b = precision_score(y_test, y_pred_pond, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision9b)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud9b = accuracy_score(y_test, y_pred_pond)
print('Exactitud del modelo:')
print(exactitud9b)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad9b = recall_score(y_test, y_pred_pond, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad9b)

Precisión del modelo:
0.24208860759493672
Exactitud del modelo:
0.9203340828782525
Sensibilidad del modelo:
0.9209048877146632


review_scores_location

In [183]:
# VarDep:'location_bin
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['review_scores_rating', 'review_scores_value']]
Var_Dep= df_log['location_bin']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [184]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  18  100]
 [  20 6088]]


In [185]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision10 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision10)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud10 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud10)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad10 = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad10)

Precisión del modelo:
0.9838396897220426
Exactitud del modelo:
0.9807259877931256
Sensibilidad del modelo:
0.15254237288135594


Reponderación de clases 

In [186]:
#Reponderación de las clases
algoritmo_Pond = LogisticRegression(class_weight='balanced')
algoritmo_Pond.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [187]:
#Realizamos una predicción
y_pred_pond = algoritmo_Pond.predict(X_test) 

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred_pond)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  89   29]
 [ 568 5540]]


In [188]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision10b = precision_score(y_test, y_pred_pond, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision10b)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud10b = accuracy_score(y_test, y_pred_pond)
print('Exactitud del modelo:')
print(exactitud10b)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad10b = recall_score(y_test, y_pred_pond, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad10b)

Precisión del modelo:
0.1354642313546423
Exactitud del modelo:
0.9041117892707998
Sensibilidad del modelo:
0.9070072036673216


In [189]:
# Crear una tabla comparativa de los resultados de los 10 modelos de regresión logística

resultados = {
    'Modelo': [
        'tiempo_respuesta', 'tasa_respuesta', 'tasa_aceptacion', 'superanfitrion',
        'distrito', 'capacidad', 'rating', 'rating_pond', 'accuracy', 'accuracy_pond', 'cleanliness', 'cleanliness_pond', 'location', 'location_pond'
    ],
    'Precisión': [
        precision1, precision2, precision3, precision4, precision5,
        precision6, precision7, precision7b, precision8, precision8b, precision9, precision9b, precision10, precision10b
    ],
    'Exactitud': [
        exactitud1, exactitud2, exactitud3, exactitud4, exactitud5,
        exactitud6, exactitud7, exactitud7b, exactitud8, exactitud8b, exactitud9, exactitud9b, exactitud10, exactitud10b
    ],
    'Sensibilidad': [
        sensibilidad1, sensibilidad2, sensibilidad3, sensibilidad4, sensibilidad5,
        sensibilidad6, sensibilidad7, sensibilidad7b, sensibilidad8, sensibilidad8b, sensibilidad9, sensibilidad9b, sensibilidad10, sensibilidad10b
    ]
}

tabla_resultados = pd.DataFrame(resultados)
tabla_resultados

,Modelo,Precisión,Exactitud,Sensibilidad
0,tiempo_respuesta,0.825243,0.807260,0.261745
1,tasa_respuesta,0.969067,0.958240,0.427692
2,tasa_aceptacion,0.900788,0.897687,0.265509
3,superanfitrion,0.466867,0.725345,0.960996
4,distrito,0.586285,0.615483,0.847627
5,capacidad,0.826133,0.798587,0.913043
6,rating,0.988519,0.985544,0.608939
7,rating_pond,0.336557,0.944105,0.943278
8,accuracy,0.988404,0.985063,0.532895
9,accuracy_pond,0.315098,0.948442,0.948469
